In [ ]:
# !pip install opencv-python
# !pip install pyserial

## Add Fire Button to Pure OPen CV

In [ ]:
import tkinter as tk
import serial
import time
import cv2
import numpy as np


serialcomm = serial.Serial(
    port="COM3", baudrate=9600
    # port="/dev/ttyACM0", baudrate=9600
    , bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE
)

# serialcomm.timeout = 1

def upturn():
    i="u"    
    serialcomm.write(i.encode())

def downturn():
    i="d"    
    serialcomm.write(i.encode())

def leftturn():
    i="l"    
    serialcomm.write(i.encode())

def rightturn():
    i="r"    
    serialcomm.write(i.encode())
    
def fire():
    i="f"    
    time.sleep(0.2)
    serialcomm.write(i.encode())

#OpenCV load model and instantate that model
net = cv2.dnn.readNet("dnn_small/yolov4-tiny.weights","dnn_small/yolov4-tiny.cfg")
model = cv2.dnn_DetectionModel(net)
model.setInputParams(size=(320,320),scale=1/255)


#initialize camera
cap = cv2.VideoCapture(0)
# cap.set(cap.CAP_PROP_FRAME_WIDTH, 1280)
# cap.set(cap.CAP_PROP_FRAME_HEIGHT, 720)



button_person = False
button_book = False
button_laptop = False


def click_button(event, x, y, flags, params):
    global button_person, button_book, button_laptop
    if event == cv2.EVENT_LBUTTONDOWN:
#         print(x,y)
        poly = np.array([[(20,20),(220,20),(220,70),(20,70)]])
        poly_book = np.array([[(20,80),(220,80),(220,130),(20,130)]])
        poly_laptop = np.array([[(20,140),(220,140),(220,190),(20,190)]])
        poly_fire = np.array([[(20,200),(220,140),(220,250),(20,250)]])
        
        is_inside =  cv2.pointPolygonTest(poly, (x,y), False)
        is_inside_book =  cv2.pointPolygonTest(poly_book, (x,y), False)
        is_inside_laptop =  cv2.pointPolygonTest(poly_laptop, (x,y), False)
        is_inside_fire =  cv2.pointPolygonTest(poly_fire, (x,y), False)
        
        if is_inside > 0:
            if button_person == False:
                button_person=True
            else:
                button_person = False
                
                    
        if is_inside_book > 0:       
            if button_book == False:
                button_book=True
            else:
                button_book = False
                
        if is_inside_laptop > 0:       
            if button_laptop == False:
                button_laptop=True
            else:
                button_laptop = False
                
        if is_inside_fire > 0: 
#                 clickfire
            fire()
            print("FIRED")
    
        print("Butt Person Stat: ",button_person )
        print("Butt Book Stat: ",button_book )
        print("Butt Laptop Stat: ",button_laptop )
        
    
def track_target(x,y):
            x_center = x+w//2
            y_center = y+h//2
            
            if   x_center < (hor_ref + w//10):
                leftturn()
                # print(">>>")
            elif  x_center > (hor_ref - w//10):
                rightturn()
                # print("<<<")
                
            if  y_center < ver_ref+h//10:
                downturn()
                # print("up")
            elif   y_center > ver_ref-h//10:
                upturn()
                # print("down")

#create Window
cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", click_button)



#load coco labels
object_lbl = []
with open("dnn_small/coco_labels.txt", "r") as file_object:
    for class_name in file_object.readlines():
        class_name=class_name.strip()
        object_lbl.append(class_name)
# print(object_lbl)

while True:
    ret, frame = cap.read()
    
#     resize frame to something bigger
    frame = cv2.resize(frame, (0, 0),fx=1.7,fy=1.7)
    
    # check for center of capture
    rad = 50
    (h_frm, w_frm) = frame.shape[:2] #w:image-width and h:image-height
    cv2.circle(frame, (w_frm//2, h_frm//2), rad, (255, 255, 255), 1) 
    hor_ref = w_frm//2 # Horizontal reference
    ver_ref = h_frm//2 # vertical reference
    
    #object Detection
    (class_ids,scores,bboxes) = model.detect(frame)
    for class_id, score, bbox in zip(class_ids, scores, bboxes):
        (x, y, w, h) = bbox
        class_name = object_lbl[class_id]
        
        if class_name == "person" and button_person==True:
    #         color = colors[class_id]
            cv2.putText(frame, class_name, (x,y-5), cv2.FONT_HERSHEY_PLAIN, 1, (200,0,50),2)
#             cv2.putText(frame, str(x)+","+str(y), (x,y-5), cv2.FONT_HERSHEY_PLAIN, 1, (200,0,50),2)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (200,0,50), 3)
            # draw cross hairs
            cv2.line(frame, (x+w//3,y+h//2), (x+2*w//3, y+h//2), (0,0,255), 2)
            cv2.line(frame, (x+w//2,y+h//3), (x+w//2, y+2*h//3), (0,0,255), 2)
        
            track_target(x,y)
               
            
            
        
        if class_name in("clock","book","scissors","knife","cup","wine glass","potted plant") and button_book==True:
    #         color = colors[class_id]
            cv2.putText(frame, class_name, (x,y-5), cv2.FONT_HERSHEY_PLAIN, 1, (200,0,50),2)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (200,0,50), 3)

            # draw cross hairs
            cv2.line(frame, (x+w//3,y+h//2), (x+2*w//3, y+h//2), (0,0,255), 2)
            cv2.line(frame, (x+w//2,y+h//3), (x+w//2, y+2*h//3), (0,0,255), 2)
            
            track_target(x,y)
            
        if class_name in ("cell phone", "laptop","remote") and button_laptop==True:
    #         color = colors[class_id]
            cv2.putText(frame, class_name, (x,y-5), cv2.FONT_HERSHEY_PLAIN, 1, (200,0,50),2)
#             cv2.putText(frame, str(x)+","+str(y), (x,y-5), cv2.FONT_HERSHEY_PLAIN, 1, (200,0,50),2)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (200,0,50), 3)
            # draw cross hairs
            cv2.line(frame, (x+w//3,y+h//2), (x+2*w//3, y+h//2), (0,0,255), 2)
            cv2.line(frame, (x+w//2,y+h//3), (x+w//2, y+2*h//3), (0,0,255), 2)
            track_target(x,y)
        


#     #create buttons
    cv2.rectangle(frame, (20,20), (150,70), (0,0,200), -1)
    cv2.putText(frame, "Person", (30,60), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3)
    cv2.rectangle(frame, (20,80), (240,130), (0,0,240), -1)
    cv2.putText(frame, "house thing", (30,120), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3)
    cv2.rectangle(frame, (20,140), (150,190), (0,0,200), -1)
    cv2.putText(frame, "device", (30,180), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3)
    cv2.rectangle(frame, (20,200), (100,240), (200,0,200), -1)
    cv2.putText(frame, "FIRE", (30,230), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3)
        

    
    
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
serialcomm.close()

Butt Person Stat:  False
Butt Book Stat:  False
Butt Laptop Stat:  True
Butt Person Stat:  False
Butt Book Stat:  False
Butt Laptop Stat:  False
Butt Person Stat:  True
Butt Book Stat:  False
Butt Laptop Stat:  False
Butt Person Stat:  False
Butt Book Stat:  False
Butt Laptop Stat:  False
Butt Person Stat:  False
Butt Book Stat:  True
Butt Laptop Stat:  False


In [ ]:
# import cv2
# print (cv2.__version__)
serialcomm.close()

In [ ]:
import tkinter as tk
import serial
import time

serialcomm = serial.Serial(
    port="COM3", baudrate=9600
    # port="/dev/ttyACM0", baudrate=9600
    , bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE
)

serialcomm.timeout = 1

def upturn():
    i="u"    
    serialcomm.write(i.encode())

def downturn():
    i="d"    
    serialcomm.write(i.encode())

def leftturn():
    i="l"    
    serialcomm.write(i.encode())

def rightturn():
    i="r"    
    serialcomm.write(i.encode())
    
def fire():
    i="f"    
    time.sleep(0.2)
    serialcomm.write(i.encode())
    

root = tk.Tk()
root.title("RoboGun Control")
btn_up=tk.Button(root,text="^",command=upturn, bg="green")
btn_up.grid(row=1,column=3, sticky="n")

btn_lft=tk.Button(root,text="<",command=leftturn, bg="green")
btn_lft.grid(row=2,column=2, sticky="w")

btn_rgh=tk.Button(root,text=">",command=rightturn, bg="green")
btn_rgh.grid(row=2,column=4, sticky="e")

btn_dwn=tk.Button(root,text="v",command=downturn, bg="green")
btn_dwn.grid(row=3,column=3, sticky="s")

btn_fire=tk.Button(root,text="FIRE",command=fire, bg="red")
btn_fire.grid(row=2,column=3, sticky="s")

root.geometry("300x300")
root.mainloop()
serialcomm.close()